## PromptTemplate


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


LLM 객체를 정의합니다. 

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 정의합니다.

In [5]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

`country` 변수에 값을 넣어서 문장을 생성할 수 있습니다.

In [6]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [8]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성


추가 유효성 검사를 위해 `input_variables` 를 명시적으로 지정하세요.

이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킵니다.

In [8]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [9]:
# prompt 생성
prompt.format(country="영국")

'영국의 수도는 어디인가요?'

In [10]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
prompt.format(country1="아르헨티나")

'아르헨티나과 미국의 수도는 각각 어디인가요?'

In [12]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [13]:
prompt_partial.format(country1="칠레")

'칠레과 캐나다의 수도는 각각 어디인가요?'

In [14]:
chain = prompt_partial | llm

In [15]:
chain.invoke("칠레").content

'칠레의 수도는 사티아고이고, 캐나다의 수도는 오타와입니다.'

In [16]:
chain.invoke({"country1": "뉴질랜드", "country2": "호주"}).content

'뉴질랜드의 수도는 웰링턴이며, 호주의 수도는 캔버라입니다.'

### `partial_variables`: 부분 변수 채움

`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. 이 사용 사례는 **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우입니다.

대표적인 예가 **날짜나 시간** 입니다.

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 **날짜를 반환하는 함수** 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.


다음의 코드는 오늘 날짜를 구하는 파이썬 코드입니다.

In [17]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'July 02'

In [18]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [19]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요. 그들의 직업과 업적에 대해 같이 설명해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [25]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 July 02 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [26]:
# chain 을 생성합니다.
chain = prompt | llm

In [27]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3).content)

1. 리브 타일러 (Liv Tyler) - 1977년 7월 1일
2. 리처드 톰슨 (Richard Thompson) - 1949년 7월 3일
3. 엘리자베스 울리드지 (Elizabeth Wulridg) - 1961년 7월 2일


In [28]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke({"today": "Mar 20", "n": 5}).content)

1. 폴 카더모니 (Paul Kaldormony) - 1926년 3월 20일
2. 카를로스 퀸텔 (Carlos Quintal) - 1963년 3월 20일
3. 에밀리아 클라크 (Emilia Clarke) - 1987년 3월 20일
4. 비안카 안헬 (Bianca Anhel) - 1991년 3월 20일
5. 린제이 본 (Lindsay Vaughn) - 1984년 3월 20일


## 파일로부터 template 읽어오기


In [29]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(input_variables=['fruit'], template='{fruit}의 색깔이 뭐야?')

Window 사용자 중 이전의 코드가 오류가 나는 경우 아래의 코드로 실행하세요(인코딩 설정)

In [ ]:
# from langchain_teddynote.prompts import load_prompt

# # Windows 사용자 only: 인코딩을 cp949로 설정
# load_prompt("prompts/fruit_color.yaml", encoding="cp949")

In [30]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [31]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



## ChatPromptTemplate

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있습니다.

**role**
- `"system"`: 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
- `"human"` : 사용자 입력 메시지 입니다.
- `"ai"`: AI 의 답변 메시지입니다.

In [32]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [33]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [34]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="윤총", user_input="당신의 이름과 나이는 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 윤총 입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름과 나이는 무엇입니까?')]

생성한 메시지를 바로 주입하여 결과를 받을 수 있습니다.

In [35]:
llm.invoke(messages).content

'제 이름은 윤총이고, 제 나이는 없습니다. 제가 AI 어시스턴트이기 때문에 나이 개념이 적용되지 않아요. 어떤 것을 도와드릴까요?'

이번에는 체인을 생성해 보겠습니다.

In [36]:
chain = chat_template | llm

In [37]:
chain.invoke({"name": "Teddy", "user_input": "당신의 성별은 무엇입니까?"}).content

'전 성별이 없는 AI 어시스턴트입니다. 어떤 질문이든지 답변해드릴 수 있어요. 어떤 도움이 필요하세요?'

## MessagePlaceholder

또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공합니다. 

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

`conversation` 대화목록을 나중에 추가하고자 할 때 `MessagesPlaceholder` 를 사용할 수 있습니다.

In [40]:
chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

'Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁 드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'

In [41]:
# chain 생성
chain = chat_prompt | llm

In [42]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

AIMessage(content='새로 입사한 테디 만남 반가워요!', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 90, 'total_tokens': 108}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9dc76340-ea99-4083-81b6-d1f504afdc35-0', usage_metadata={'input_tokens': 90, 'output_tokens': 18, 'total_tokens': 108})